# Run Kohyaa in runpod
# NoteBook Created by Voiid (https://github.com/official-imvoiid)
📌 Recommended Setup:
- GPU: A6000 Ada  
- Storage: 70GB total  
  - 50GB on persistent  
  - 20GB on temporary

DIRECT MODEL DOWNLOAD VIA HUGGINGFACE IF YOU WANNA FINETUNE ON CUSTOM MODEL (OPTIONAL)

In [ ]:
# Install required library
!pip install huggingface_hub

from huggingface_hub import login, snapshot_download
import os

# Prompt for Hugging Face access token
hf_token = input("Enter your Hugging Face access token: ")

# Log in to Hugging Face
login(hf_token)

# Define model name and download directory
model_name = input("Enter the Hugging Face model name (e.g., OnomaAIResearch/Illustrious-XL-v1.1): ")
download_dir = "/workspace/Model/HuggingFace/"

# Create download directory if it doesn't exist
os.makedirs(download_dir, exist_ok=True)

# Download the model
try:
    snapshot_download(repo_id=model_name, local_dir=download_dir, allow_patterns="*")
    print(f"Model {model_name} downloaded successfully to {download_dir}")
except Exception as e:
    print(f"Error downloading model: {e}")

# List downloaded files
if os.path.exists(download_dir):
    print("\nFiles in the download directory:")
    for root, dirs, files in os.walk(download_dir):
        for file in files:
            print(os.path.join(root, file))

DIRECT MODEL DOWNLOAD VIA CIVITAI IF YOU WANNA FINETUNE ON CUSTOM MODEL (OPTIONAL)

In [ ]:
# CivitAi Model Downloader Via API

import os
import re
import sys
import requests
from tqdm import tqdm

# Set your CivitAI API key here
API_KEY = "API_KEY"  # ← set your CivitAI key

# ─── Paste your URLs here ────────────────────────────────────────────────
# Base directory paths
CKPT_PATH = "/workspace/Models/Civitai"


ckpt_links = [
    "https://civitai.com/models/257749/pony-diffusion-v6-xl",
    "https://civitai.com/models/795765/illustrious-xl",
    "https://civitai.com/models/101055/sd-xl"
]


# ──────────────────────────────────────────────────────────────────────────
# Check if API key is set
if not API_KEY:
    print("API key not set. Please set your CivitAI API key.")
    sys.exit(1)

# Create all required directories
os.makedirs(CKPT_PATH, exist_ok=True)

def parse_mid_and_override(url):
    m = re.search(r"models/(\d+)", url)
    if not m:
        raise ValueError(f"Bad URL: {url}")
    mid = m.group(1)
    over = re.search(r"modelVersionId=(\d+)", url)
    return mid, (over.group(1) if over else None)

def get_vid(mid):
    h = {"Authorization": f"Bearer {API_KEY}"}
    r = requests.get(
        f"https://civitai.com/api/v1/models/{mid}", headers=h, timeout=10
    )
    r.raise_for_status()
    return r.json()["modelVersions"][0]["id"]

def fetch_dl_info(vid):
    h = {"Authorization": f"Bearer {API_KEY}"}
    r = requests.get(
        f"https://civitai.com/api/v1/model-versions/{vid}", headers=h, timeout=10
    )
    r.raise_for_status()
    j = r.json()
    return j["downloadUrl"], j["files"][0]["name"]

def download(url, dest_folder):
    try:
        mid, override = parse_mid_and_override(url)
        vid = override or get_vid(mid)
        dl_url, fname = fetch_dl_info(vid)
        print(f"Processing: {url}")
        print(f"Model ID: {mid}, Version ID: {vid}")
        print(f"Filename: {fname}")
    except Exception as e:
        print(f"Error processing URL {url}: {e}")
        return

    path = os.path.join(dest_folder, fname)

    if os.path.exists(path):
        print(f"File already exists: {path}")
        return

    print(f"Downloading to: {path}")

    try:
        r = requests.get(
            dl_url,
            headers={"Authorization": f"Bearer {API_KEY}"},
            stream=True
        )

        if r.status_code != 200:
            print(f"Download failed with status code {r.status_code}")
            return

        total = int(r.headers.get("content-length", 0))
        bar = tqdm(total=total, unit="iB", unit_scale=True, desc=fname)

        with open(path, "wb") as f:
            for chunk in r.iter_content(8192):
                f.write(chunk)
                bar.update(len(chunk))

        bar.close()
        print(f"Downloaded: {fname}")
    except Exception as e:
        print(f"Error downloading {fname}: {e}")
        if os.path.exists(path):
            os.remove(path)  # Remove partially downloaded file

# ─── Kick off downloads ───────────────────────────────────────────────────
print("Starting downloads...")

print("\nDownloading checkpoint models...")
for u in ckpt_links:
    download(u, CKPT_PATH)

print("\n✨ All downloads completed.")

CLONE KOHYA_SS REPO & INSTALL REQUIRNMENTS

NOTE: THIS SCRIPT EXICUTES IN CMD AT BACKGROUND THIS MIGHT TAKE A FEW MINS
SINCE THIS RUN IN BACKGROUND CMD YOU CAN CHECK LIVE PROGRESS AT workspace/kohya_ss/logs/setup/kohya_ss_gui_.log
TOOK ME ABOUT 10 MINS TO INSTALL BEFORE IT WAS READY TO USE

In [ ]:
import os
import subprocess
import sys

def run_command(command, cwd=None):
    """Run a shell command and return the result"""
    try:
        result = subprocess.run(
            command, 
            shell=True, 
            check=True, 
            capture_output=True, 
            text=True,
            cwd=cwd
        )
        print(f"✓ Command executed successfully: {command}")
        if result.stdout:
            print(f"Output: {result.stdout}")
        return result
    except subprocess.CalledProcessError as e:
        print(f"✗ Error executing command: {command}")
        print(f"Error: {e.stderr}")
        return None

# Step 1: Change to workspace directory
print("Step 1: Changing to /workspace directory...")
try:
    os.chdir('/workspace')
    print(f"✓ Current directory: {os.getcwd()}")
except FileNotFoundError:
    print("✗ /workspace directory not found. Using current directory instead.")
    print(f"Current directory: {os.getcwd()}")

# Step 2: Clone the repository with recursive submodules
print("\nStep 2: Cloning kohya_ss repository...")
clone_command = "git clone --recursive https://github.com/bmaltais/kohya_ss.git"
clone_result = run_command(clone_command)

if clone_result:
    # Step 3: Change to kohya_ss directory
    print("\nStep 3: Changing to kohya_ss directory...")
    try:
        os.chdir('kohya_ss')
        print(f"✓ Current directory: {os.getcwd()}")
        
        # Step 4: Run the setup script
        print("\nStep 4: Running setup-runpod.sh...")
        
        # First, make sure the script is executable
        run_command("chmod +x setup-runpod.sh")
        
        # Run the setup script
        setup_result = run_command("./setup-runpod.sh")
        
        if setup_result:
            print("\n🎉 Setup completed successfully!")
        else:
            print("\n❌ Setup script failed. Check the error messages above.")
            
    except FileNotFoundError:
        print("✗ kohya_ss directory not found after cloning.")
else:
    print("❌ Failed to clone repository. Please check your internet connection and try again.")

# Optional: List contents of current directory to verify
print(f"\nContents of current directory ({os.getcwd()}):")
try:
    contents = os.listdir('.')
    for item in contents:
        print(f"  - {item}")
except Exception as e:
    print(f"Error listing directory contents: {e}")

START KOHYA_SS WEBUI

In [ ]:
import os
os.chdir('/workspace/kohya_ss')

!./gui.sh --share

Activating venv...
2025-06-18 19:13:46.676228: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-18 19:13:46.676321: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-18 19:13:46.676385: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-18 19:13:46.686632: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-18 19:13:51.622044: W t